In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**plotlyを使ったEDA**  
kaggle初心者がやった勉強のための分析。
<br>
<br>
参考notebook
- https://www.kaggle.com/ekrembayar/store-sales-ts-forecasting-a-comprehensive-guide/notebook
- https://www.kaggle.com/shivamb/store-sales-forecasting-exploration/notebook

In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.linear_model import LinearRegression
from pandas import date_range
from statsmodels.graphics.tsaplots import plot_pacf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
df_holidays = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv', header = 0)
df_oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv', header = 0)
df_stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv', header = 0)
df_transactions = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv', header = 0)

train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv', header = 0)
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv', header = 0)

In [ ]:
df_holidays['date'] = pd.to_datetime(df_holidays['date'], format = "%Y-%m-%d")
df_oil['date'] = pd.to_datetime(df_oil['date'], format = "%Y-%m-%d")
df_transactions['date'] = pd.to_datetime(df_transactions['date'], format = "%Y-%m-%d")
train['date'] = pd.to_datetime(train['date'], format = "%Y-%m-%d")
test['date'] = pd.to_datetime(test['date'], format = "%Y-%m-%d")

In [ ]:
df_transactions.head()

In [ ]:
print("Trainng Data Shape: ", train.shape)
print("Tresting Data Shape: ", test.shape)
train.head()

In [ ]:
test.head()

In [ ]:
train1 = train.merge(df_oil, on='date', how='left')
train1.head(10)

In [ ]:
df_stores.head(10)

In [ ]:
df_holidays.head()

In [ ]:
df_transactions.head(10)
print(df_transactions.shape)

In [ ]:
store = df_transactions.merge(df_stores, on='store_nbr', how='right')
store.head(20)

In [ ]:
train1 = train1.merge(df_holidays, on='date', how='left')
train1 = train1.merge(df_stores, on='store_nbr', how='left')
train1 = train1.merge(df_transactions, on=['date', 'store_nbr'], how='left')

train1 = train1.rename(columns={"type_x":"holiday_type", "type_y":"store_type"})

train1.head()

In [ ]:
agg = train1.groupby('date').agg({"sales":"mean"}).reset_index()
agg.head()

とりあえず、売り上げや客数の推移あたりから見る

In [ ]:
agg = train1.groupby('date').agg({"sales":"mean"}).reset_index()
fig = px.line(agg, x="date", y="sales")
fig.update_layout(title="Average Sales by Date")
fig.show()

agg = train1.groupby('date').agg({"transactions":"mean"}).reset_index()
fig = px.line(agg, x="date", y="transactions")
fig.update_layout(title="Average Transactions by Date")
fig.show()

## 上のグラフから
- トランザクション数の推移は横ばい。しかし、売上は右肩上がり。
    - 単価が上がっている？
- クリスマスの前日は客数が極端に多いのに売上は上がっていない。
    - セールを開催しているのかも。
- 1/1は売上0
    - 新年は休み
1. トランザクション → 客数
2. sales → 売上高

## store_typeと売上・客数との相関を調べて折れ線グラフにする

In [ ]:
agg = train1.groupby(['date', 'store_type']).agg({"sales":"mean"}).reset_index()
fig = px.line(agg, x="date", y="sales", color="store_type")
fig.update_layout(title="Average Sales by Date and Store Type")
fig.show()

agg = train1.groupby(['date', 'store_type']).agg({"transactions":"mean"}).reset_index()
fig = px.line(agg, x="date", y="transactions", color="store_type")
fig.update_layout(title="Average Transactions by Date and Store Type")
fig.show()

## 上のグラフより
- store_type 「B」はtransactionは多いのにsalesは低い
    - 客数は多いのに、売上に繋がっていない可能性がある
- store_type 「D」はBに比べてtransactionが少ないのにsalesがBを上回る期間が多い
    - Dの単価は高い？

## clusterと売上・客数との相関を調べて折れ線グラフにする

In [ ]:
agg = train1.groupby(['date', 'cluster']).agg({"sales" : "mean"}).reset_index()
fig = px.line(agg, x='date', y="sales", color='cluster')
fig.update_layout(title = "Average Sales by Date and Cluster")
fig.show()


agg = train1.groupby(['date', 'cluster']).agg({"transactions" : "mean"}).reset_index()
fig = px.line(agg, x='date', y="transactions", color='cluster')
fig.update_layout(title = "Average Transactions by Date and Cluster")
fig.show()

## 上図より
- 各パラメーター間の関係性がよく分からない
- もう少し関係性の詳細度を高めたい

In [ ]:
train1['family'].shape

In [ ]:
print(train1["cluster"].unique())
print(train1["store_nbr"].unique())

## 売上平均と他のパラメーターの相関性を調べて棒グラフにする

In [ ]:
# 棒グラフ作成のためのパラメーター関数(売り上げ)

def vbar(col):
    temp = train1.groupby(col).agg({"sales":"mean"}).reset_index()
    temp = temp.sort_values('sales', ascending=False)

    param = {
        'x':list(temp["sales"])[:][::-1],
        'y':list(temp[col])[:][::-1],
        'title':"Average sales by" + col
    }

    bar = go.Bar(y=[str(_) + " " for _ in param['y']],
               x=param['x'],
               orientation="h",
               marker=dict(color="skyblue"))
    return bar

    layout = go.Layout(title=params['title'],
                      paper_bgcolor='rgba(0,0,0,0)',
                       plot_bgcolor='rgba(0,0,0,0)',
                       xaxis_title="", 
                       yaxis_title="", 
                       width=650)

    fig = go.Figure([trace], layout=layout)

    # layout = go.Layout(title="Average sales by Family",
    #                   paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
    #                    xaxis_title="", yaxis_title="", width=650)

    fig.update_xaxes(tickangle=0, tickfont=dict(color='black'))
    fig.update_yaxes(tickangle=0, tickfont=dict(color='black'))
    fig.show()

In [ ]:
bar1 = vbar("family")
bar2 = vbar("state")
bar3 = vbar("store_type")
bar4 = vbar("city")
bar5 = vbar("cluster")
bar6 = vbar("store_nbr")

titles = ["Store Family", "State", "Store Type", "City", "Cluster Number", "Store Number"]
titles = ["Top " + _ + "by Average Sales " for _ in titles]

fig = make_subplots(rows=3, cols=2, subplot_titles=titles)

fig.add_trace(bar1, row=1, col=1)
fig.add_trace(bar2, row=1, col=2)
fig.add_trace(bar3, row=2, col=1)
fig.add_trace(bar4, row=2, col=2)
fig.add_trace(bar5, row=3, col=1)
fig.add_trace(bar6, row=3, col=2)

fig.update_layout(height=2000,
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  showlegend = False)

# fig.update_layout(barmode='group')
fig.show()

## 上グラフより
- 一番売れているのはgrocery1
- 各パラメーターの売上別ランクが分かったので、次は高ランク同士での相関関係を調べたい。
    - 例えば、top_cityで最も売上高の高い店舗が売っている商品種別やトランザクション数を調べるなど→効率・商品単価など分かりそう
- 全く売れてなさそうな品目がある。
    - モデル構築の際に切り捨てるか？
- familyとstoreなどの関係はありそうか？

In [ ]:
train1.head()

In [ ]:
#売り上げなしの品目を調べる
def r_family_0(col):
    a = train1.groupby([col, "family"]).sales.sum().reset_index().sort_values(["family", col])
    a = a[a.sales == 0]
    print(a.shape)
    print(a)

In [ ]:
titles = ["state","store_type","city","cluster", "store_nbr"]

for title in titles:
    print(r_family_0(title))

## トランザクションと他のパラメーターの関係

In [ ]:
# 棒グラフ作成のためのパラメーター関数(客数)

def tbar(col):
    temp = train1.groupby(col).agg({"transactions":"mean"}).reset_index()
    temp = temp.sort_values('transactions', ascending=False)

    param = {
        'x':list(temp["transactions"])[:][::-1],
        'y':list(temp[col])[:][::-1],
        'title':"Average transactions by" + col
    }

    bar = go.Bar(y=[str(_) + " " for _ in param['y']],
               x=param['x'],
               orientation="h",
               marker=dict(color="palegreen"))
    return bar

    layout = go.Layout(title=params['title'],
                      paper_bgcolor='rgba(0,0,0,0)',
                       plot_bgcolor='rgba(0,0,0,0)',
                       xaxis_title="", 
                       yaxis_title="", 
                       width=650)

    tfig = go.Figure([trace], layout=layout)

    tfig.update_xaxes(tickangle=0, tickfont=dict(color='black'))
    tfig.update_yaxes(tickangle=0, tickfont=dict(color='black'))
    tfig.show()

In [ ]:
bar1 = tbar("state")
bar2 = tbar("store_type")
bar3 = tbar("city")
bar4 = tbar("cluster")
bar5 = tbar("store_nbr")

titles = ["State", "Store Type", "City", "Cluster Number", "Store Number"]
titles = ["Top " + _ + "by Average Transactions " for _ in titles]

tfig = make_subplots(rows=3, cols=2, subplot_titles=titles)

tfig.add_trace(bar1, row=1, col=1)
tfig.add_trace(bar2, row=1, col=2)
tfig.add_trace(bar3, row=2, col=1)
tfig.add_trace(bar4, row=2, col=2)
tfig.add_trace(bar5, row=3, col=1)

tfig.update_layout(height=2000,
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  showlegend = False)

# fig.update_layout(barmode='group')
tfig.show()

- 結局ごちゃごちゃして分からなくなってしまったので、折れ線グラフの方が関係性が分かりそう。

## というわけで、折れ線グラフに戻る

In [ ]:
agg = train1.groupby(['date', 'store_type']).agg({"sales":"mean"}).reset_index()
fig = px.line(agg, x="date", y="sales", color="store_type")
fig.update_layout(title="Average Sales by Date and Store Type")
fig.show()

agg = train1.groupby(['date', 'store_type']).agg({"transactions":"mean"}).reset_index()
fig = px.line(agg, x="date", y="transactions", color="store_type")
fig.update_layout(title="Average Transactions by Date and Store Type")
fig.show()

agg = train1.groupby(['date', 'state']).agg({"sales":"mean"}).reset_index()
fig = px.line(agg, x="date", y="sales", color="state")
fig.update_layout(title="Average Sales by Date and State")
fig.show()

agg = train1.groupby(['date', 'state']).agg({"transactions":"mean"}).reset_index()
fig = px.line(agg, x="date", y="transactions", color="state")
fig.update_layout(title="Average Transactions by Date and State")
fig.show()


- Pastazaの売り上げが2016年まで０→新規開店の店舗かな
- 上位stateの売り上げとトランザクションのランクはほぼ一致
    - 客数が多いと売り上げが上がるのは鉄板
    - 揃えているグッズがいいのか、立地がいいのか、店舗自体大きいのか
        - 売れている品目はダントツで生鮮食品なので、スーパーに近い感じなのかも。
        - 客数が横ばい、売り上げが上がる→値上げ？
            - 輸送コスト？原材料の高騰？気候変動？→使える情報はoilなのでこっち調べるとなんか分かりそう。
    - ただ、santa elenaに関して、トランザクションが下から二番目なのに売上は下から五番目ぐらい

In [ ]:
agg = train1.groupby(['date', 'store_nbr']).agg({"sales":"mean"}).reset_index()
fig = px.line(agg, x="date", y="sales", color="store_nbr")
fig.update_layout(title="Average Sales by Date and store_nbr")
fig.show()

agg = train1.groupby(['date', 'store_nbr']).agg({"transactions":"mean"}).reset_index()
fig = px.line(agg, x="date", y="transactions", color="store_nbr")
fig.update_layout(title="Average Transactions by Date and store_nbr")
fig.show()

agg = train1.groupby(['date', 'city']).agg({"sales":"mean"}).reset_index()
fig = px.line(agg, x="date", y="sales", color="city")
fig.update_layout(title="Average Sales by Date and city")
fig.show()

agg = train1.groupby(['date', 'city']).agg({"transactions":"mean"}).reset_index()
fig = px.line(agg, x="date", y="transactions", color="city")
fig.update_layout(title="Average Transactions by Date and city")
fig.show()

- cityとstore_nbrで見ると売上のグラフは右肩上がりではなく、売上低いところは割とそのまま→グラフの総面積が増えている感じ
    - 店舗のタイプ別に見ると、売り上げが底上げされていると感じたが、元々売り上げを上げている店舗の売り上げがさらに上がったという感じ？

## Oil関連のデータを見てみる

In [ ]:
df_oil.head()

In [ ]:
df_oil.isnull().sum()

- 欠損値が存在するので、何かで補完したい
    - time series系には前後の値から保管してくれるinterpolationが良さそうとの記事があったので、それを試してみる

In [ ]:
df_oil['date'] = pd.to_datetime(df_oil.date)

df_oil = df_oil.set_index("date").dcoilwtico.resample("D").sum().reset_index()
df_oil['dcoilwtico'] = np.where(df_oil['dcoilwtico'] == 0, np.nan, df_oil['dcoilwtico'])
df_oil['dcoilwtico_interporated'] = df_oil.dcoilwtico.interpolate()

df_oil.head()

In [ ]:
print(df_oil.keys()[5:])

In [ ]:
p = df_oil.melt(id_vars=['date']+list(df_oil.keys()[3:]), var_name='Legend')
print(p.head())
px.line(p.sort_values(["Legend", "date"], ascending = [False, True]), x='date', y='value', color='Legend',title = "Daily Oil Price" )

- oilの価格は下がっている
    - 原油価格が上昇して、物価が上がったというストーリーは崩れる
    - そもそも原油価格の推移と売り上げの推移を一緒に見ないと結論はでない